In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv


### Reading ratings data

In [2]:
ratings = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv', parse_dates=['timestamp'])

In [ ]:
ratings.head()

In [ ]:
ratings[['userId', 'movieId']].nunique()

In [ ]:
ratings['yearNdMonth'] = ratings['timestamp'].apply(lambda date: date.strftime('%Y-%m'))

In [ ]:
ratings['yearNdMonth'].value_counts()

In [ ]:
ratings['rating'].plot(kind='hist')

### Randomly subsampling ratings data 

In [3]:
sampledUserIds = np.random.choice(ratings['userId'].unique(), size = 5000, replace = False)

ratings = ratings[ratings['userId'].isin(sampledUserIds)]

print('There are {} rows of data from {} users'.format(len(ratings), len(sampledUserIds)))

There are 720699 rows of data from 5000 users


### Creating train/test split using timestamp information

In [4]:
ratings['rankMoviesWatched'] = ratings.groupby(['userId'])['timestamp'] \
                                .rank(method='first', ascending=False)

In [5]:
train = ratings[ratings['rankMoviesWatched'] != 1]
test = ratings[ratings['rankMoviesWatched'] == 1]

# keeping columns that we need
train = train[['userId', 'movieId', 'rating']]
test = test[['userId', 'movieId', 'rating']]

In [6]:
train.shape, test.shape

((715699, 3), (5000, 3))

### Converting rating to binary values

In [ ]:
train['rating'].value_counts()

In [7]:
train['target'] = np.where(train['rating']>=2.5, 1, 0)
test['target'] = np.where(test['rating']>=2.5, 1, 0)

In [ ]:
train['target'].value_counts()

In [8]:
train.drop(columns=['rating'], inplace=True)
test.drop(columns=['rating'], inplace=True)

In [ ]:
train.head()

### Creating deep learning based collaborative filtering model using Pytorch

In [9]:
device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [10]:
class NCF(nn.Module):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
    """
    
    def __init__(self, num_users, num_items):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        
    def forward(self, x):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(x[:,0])
        item_embedded = self.item_embedding(x[:,1])

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred

In [11]:
num_users = ratings['userId'].max()+1
num_items = ratings['movieId'].max()+1

In [12]:
num_users, num_items

(138481, 131014)

In [13]:
ncf = NCF(num_users, num_items).to(device)

In [14]:
print(ncf)

NCF(
  (user_embedding): Embedding(138481, 8)
  (item_embedding): Embedding(131014, 8)
  (fc1): Linear(in_features=16, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
)


In [15]:
trainTensor = TensorDataset(torch.from_numpy(train.values[..., :2]), torch.tensor(train.values[..., 2], dtype=torch.float).view(-1,1))
testTensor = TensorDataset(torch.from_numpy(test.values[..., :2]), torch.tensor(test.values[..., 2], dtype=torch.float).view(-1,1))

In [16]:
trainData = DataLoader(trainTensor, batch_size=128, num_workers=4, shuffle=True)

In [17]:
testData = DataLoader(testTensor, batch_size=128, num_workers=4, shuffle=True)

### Training and optimizing our model

In [18]:
def train_loop(dataloader):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = ncf(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = ncf(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(ncf.parameters(), lr=0.1)
epochs = 5

In [21]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainData)
    test_loop(testData)
print("Done!")

Epoch 1
-------------------------------
loss: 0.662524  [    0/715699]
loss: 0.373876  [12800/715699]
loss: 0.416472  [25600/715699]
loss: 0.413141  [38400/715699]
loss: 0.361692  [51200/715699]
loss: 0.353573  [64000/715699]
loss: 0.407163  [76800/715699]
loss: 0.313562  [89600/715699]
loss: 0.434389  [102400/715699]
loss: 0.376773  [115200/715699]
loss: 0.294079  [128000/715699]
loss: 0.376792  [140800/715699]
loss: 0.406332  [153600/715699]
loss: 0.391977  [166400/715699]
loss: 0.391873  [179200/715699]
loss: 0.433546  [192000/715699]
loss: 0.391961  [204800/715699]
loss: 0.490202  [217600/715699]
loss: 0.420217  [230400/715699]
loss: 0.305935  [243200/715699]
loss: 0.376928  [256000/715699]
loss: 0.392205  [268800/715699]
loss: 0.517145  [281600/715699]
loss: 0.347686  [294400/715699]
loss: 0.380614  [307200/715699]
loss: 0.499756  [320000/715699]
loss: 0.406156  [332800/715699]
loss: 0.330154  [345600/715699]
loss: 0.434550  [358400/715699]
loss: 0.334002  [371200/715699]
loss: 0.

KeyboardInterrupt: 